© Copyright, 2026 G. Schaer.

SPDX-License-Identifier: GPL-3.0-only

# PID Control of a Wheel

In this project, we will import a pre-made `condynsate` project, build a controller for it, run the project with our controller, and post-process the simulation data. The project in this notebook is a wheel on an axle. The goal is to build a controller that makes the wheel point in a desired direction by applying torque to the axle under the effect of an external disturbance.

## Importing the Project

The backend for this project has already been made for you. The backend will handle building the simulation environment, running and visualizing the simulation, applying your controller inputs, and tracking relevant data. Let's start off by importing this backend. It is a function named `run` that is stored in a Python script named `wheel.py`. 

In [ ]:
# Import the project's backend.
from wheel import run

Now that we have the backend, we can make our own controller. In the scope of this project, the controller is implemented as function that takes as arguments some information about the wheel and returns the torque to apply to the wheel.

## The Effect of Disturbance

### A PD Controller

Last time, we designed a proportional derivative (PD) controller that applies torque to a wheel on a axle to make the wheel point in a desired direction. This worked well in our simple case, but it does have some limitations. For example, suppose we were running the controller on a windy day such that there was a constant external torque on the wheel. Such uncontrolled, unmeasured inputs to a dynamic system are called disturbances. Let's run a simultion to see the effects of adding a disturbance to our wheel system. 

We will again start by building our PD controller function, but in this case, we are given two extra arguments, the integral of the wheel angle and the integral of the target angle. For now, we will ignore these terms and implement the exact same controller as before.

In [ ]:
def controller(state):
    """
    The controller function. Given some state information, it calculates a torque that
    (hopefully) makes the wheel point at a target.

    Parameters
    ----------
    state : dictionary of floats with the following keys
        angle : float
            The current angle the wheel is facing in radians
        angle_integral : float
            The integral of the wheel's angle from the start of the simulation to now
        angular_rate : float
            The current angular rate of the wheel in radians / second
        target : float
            The current target angle in radians
        target_integral : float
            The integral of the target angle from the start of the simulation to now

    Returns
    -------
    torque : float
        The torque to apply to the wheel
    """
    kp = 0.35 # The proportional gain 
    kd = 0.35 # The derivative gain
    error = (state['target'] - state['angle']) # Calculate the error
    error_derivative = -state['angular_rate'] # Calculate the derivative of the error
    torque = kp * error + kd * error_derivative # Calculate the input
    return torque

Now we are ready to run the simulation and collect data. To do this, we simply call the `run` function as pass as arguments the target wheel angle in radians and the controller function we just built. To add a disturbance, we can also set the key word argument `disturbance` to some nonzero value. Let's set the target angle to 90 degrees and add 0.10 Nm of disturbance torque to the wheel. The simulation will automatically run, apply our controller, and return to us some data collected during simulation. The data is a dictionary with the values:

`data['time']` : list of n floats

    The time, in seconds, at which each data point is collected
    
`data['angle']` : list of n floats

    The angle of the wheel, in radians, at each of the n data collection points.

`data['angle_integral']` : list of n floats

    The integral of the wheel's angle, in radian-seconds, from the start of the simulation to each of the n data collection points.

`data['angular_rate']` : list of n floats

    The angular rate of the wheel, in radians per second, at each of the n data collection points.
    
`data['target']` : list of n floats

    The target angle, in radians, at each of the n data collection points.

`data['target_integral']` : list of n floats

    The integral of the target angle, in radian-seconds, from the start of the simulation to each of the n data collection points.

`data['torque']` : list of n floats

    The torque applied to the wheel, in Newton-meters, at each of the n data collection points.

In [ ]:
# Run the simulation and collect the simulation data
data = run(1.5708, controller, disturbance=0.10)

Why didn't that work?

Because there is an unaccounted for disturbance constantly rotating the wheel counterclockwise, all torques applied by the controller counterclockwise are too large and all torques applied clockwise are too small. This results in a steady state error, which is defined as the limit of the difference between the set point and the process variable as time goes to infinty, i.e., the difference between the target angle and the wheel angle at the end of the simulation.

How might we correct this? 

We could add a term that tracks the steady state error as a function of time and modifies the torque accordingly. One such function is the integral of the steady state error.

### Analyzing the PD Controller's Failure

Let's double check our intuition with a plot of the error and torque as functions of time.

In [ ]:
# Import Numpy to assist with data analysis
import numpy as np

# Import plotting tool
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot the error and torque as function of time
plt.plot(data['time'], data['target'] - data['angle'], label='Error [rad]', lw=2.0) 
plt.plot(data['time'], data['torque'], label='Torque [Nm]', lw=2.0)
plt.plot(data['time'], data['disturbance'], label='Disturbance [Nm]', lw=2.0)
plt.legend()
plt.xlabel('Time [seconds]')
plt.axhline(c='k', lw=0.5)
plt.show()

From this plot, our suspicions from above are confirmed. Indeed the error tends to a nonzero steady state as the magnitude of the controller applied torque matches the disturbance.

Recall, 
\begin{align}
e(t) = sp - pv(t)
\end{align}
where $e$ is the error, $sp$ is the set point, $pv$ is the process variable, and $t$ is time. Calculating the integral of $e$ with respect to time, we find
\begin{align}
\int_{0}^{t}e(x)dx &= \int_{0}^{t}sp - pv(x) dx\\
\int_{0}^{t}e(x)dx &= t\cdot sp - \int_{0}^{t}pv(x) dx\\
\end{align}
So the integral of the error is the integral of the set point minus the integral of the process variable. Let's plot this now.

In [ ]:
# Plot the error, derivative of the error, and torque as function of time
plt.plot(data['time'], data['target'] - data['angle'], label='Error [rad]', lw=2.0) 
plt.plot(data['time'], data['torque'], label='Torque [Nm]', lw=2.0)
plt.plot(data['time'], data['disturbance'], label='Disturbance [Nm]', lw=2.0)
plt.plot(data['time'], data['target_integral'] - data['angle_integral'], label='Error Integral [rad-s]', lw=2.0) 
plt.legend()
plt.xlabel('Time [seconds]')
plt.axhline(c='k', lw=0.5)
plt.show()

This looks promising. We see:
1. When the error is positive, the integral of the error increases.
2. When the error is negative, the integral of the error decreases.
3. The integral of the error does not reach steady state at the end of the simulation.

This gives us a way to apply a torque in response to the steady state error. All we have to do is apply another torque that is proportional to the integral of the error. Let's implement this now.

## A Proportional Integral Derivative Controller

### Designing a PID Controller

Let's implment a PID (proportional integral derivative) controller based on our insights from above. Just like before, we will build a controller that uses information about the process variable (the wheel's angle), the set point (the target angle), and their derivatives to calculate what input (torque) to apply to the system. But in this case, we will also include the integral of the error in the torque calculation. Specically, we will set the input to be directly proportional to the error, the integral of the error, and the derivative of the error. In practice, we can implement this by multiplying the error by a constant, called the proportional gain, multiplying the integral of the error by a constant, called the integral gain, multiplying the derivative of the error by a constant, called the derivative gain, and setting the input as the sum of these three values. Let's do this now.

In [ ]:
def controller(state):
    kp = 0.35 # The proportional gain 
    ki = 0.1 # The integral gain
    kd = 0.35 # The derivative gain
    error = (state['target'] - state['angle']) # Calculate the error
    error_integral = state['target_integral'] - state['angle_integral'] # Calculate the integral of the error
    error_derivative = -state['angular_rate'] # Calculate the derivative of the error
    torque = kp * error + ki * error_integral + kd * error_derivative # Calculate the input
    return torque

### Running the PID Controller
Now we are again ready to run the simulation and collect data. To do this, we call the `run` function just like before, but this time with our new controller function.

In [ ]:
# Run the simulation and collect the simulation data
data = run(1.5708, controller, disturbance=0.10)

### Visualizing the PID Controller's Success

That seems to have done it! Now, as the simulation continues, the integral of the error gets larger and therefore so does the torque applied due to this integral. This eventually results in the applied torque counteracting the disturbance and the wheel approaching the target angle. Plotting this:

In [ ]:
# Plot the error and torque as function of time
plt.plot(data['time'], data['target'] - data['angle'], label='Error [rad]', lw=2.0) 
plt.plot(data['time'], data['torque'], label='Torque [Nm]', lw=2.0)
plt.plot(data['time'], data['disturbance'], label='Disturbance [Nm]', lw=2.0)
plt.legend()
plt.xlabel('Time [seconds]')
plt.axhline(c='k', lw=0.5)
plt.show()

Voila! As the simulation continues, the error now approaches 0 and the magnitude of the applied torque approaches the magnitude of the disturbance. This is exactly what we wanted.

## Assignment

Now that we have a working controller, we will investigate how changes to the control gains alter its behavior. To do this, we alter the controller function, rerun the simulation, and investigate the results.

Steady state error is defined as the limit of the error between a set point and the process variable as time goes to infinty. For example, in this case, we can calculate the steady state error as the error observed at the end of a long simulation. On the other hand, overshoot is defined as the maximum amount by which a signal exceeds its set point after its initial rise. In this case, overshoot is the maximum magnitude of the error observed from between the rise time and the end of the simulation.

1. Calculate the rise time, 5% settling time, steady state error, and overshoot of the previous simulation. Report your answers to the nearest tenth of a second or hundredth of a radian.

2. How do you expect changing the integral gain would change the overshoot and steady state error?

   a. Test your hypothesis by running a set of at least 3 simulations with altered integral gains and plot the results. Discuss what changes you saw to the overshoot and steady state error as a function of the integral gain.

   b. Why did these results occur? Prove your reasoning using the collected simulation data. For your simulations, you might find it helpful to increase the simulation duration. You can do this by setting the key word argument `"time"`. For example, to run the simulation for 30 seconds, you would call: `data = run(1.5708, controller, disturbance=0.10, time=30.0)`

3. How do you expect changing the disturbance value would change the 5% settling time and overshoot?

   a. Test your hypothesis by running a set of at least 5 simulations with altered disturbance values (these may be positive, 0, or negative) and plot the results. Discuss what changes you saw to the 5% settling time and overshoot as a function of the disturbance value.

   b. Why did these results occur? Prove your reasoning using the collected simulation data. Again, you might find it helpful to increase the simulation duration.

4. In this system, under the effect of 0.1 Nm of disturbance, how might you tune the control gains to reduce the overshoot while maintaining approximately the same 5% settling time?

   a. Prove your hypothesis by running two simulations, both with approximately the same 5% settling times, but one with a large overshoot and the other with a reduced overshoot.

   b. Explain your reasoning for why your changes to the control gains worked.
